## Metaprogramming

- [**The \_\_new\_\_ Method**](#the_new_method)
- [**Creation of Classes**](#creation_of_classes)
- [**Metaclasses**](#metaclasses)
- [**Metaclass Parameters**](#metaclass_parameters)
- [**Class Decorators**](#class_decorators)
- [**Metaclasses vs. Class Decorators**](#metaclasses_class_decorators)
- [**Decorator Classes**](#decorator_classes)
- [**The \_\_prepare\_\_ Method**](#the_prepare_method)

---

### The \_\_new\_\_ Method <a name='the_new_method'></a>

`__new__` method is called during creation of instances of any class (which is implemented by default in `object` class). The workflow of creating an instance is:
* \_\_new\_\_ method is called as creating a new instance of a class.
* Arguments passed into the \_\_new\_\_ method are ignored and then \_\_init\_\_ method is called with these arguments passed in (note that the type of \_\_new\_\_ method and \_\_init\_\_ method must match, otherwise \_\_init\_\_ will not be called).

In [1]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

In [2]:
# Call __new__ directly
p = object.__new__(Point)
print(type(p))
print(p.__dict__)

<class '__main__.Point'>
{}


In [3]:
p.__init__(2, 7)
print(p.__dict__)

{'x': 2, 'y': 7}


`__new__` is static, and it is possible to override `__new__` method.

In [4]:
class Person:
    def __new__(cls):
        print('Person class instantiating...')
        instance = super().__new__(cls)
        
        return instance

In [5]:
p = Person()

Person class instantiating...


---

### Creation of Classes <a name='creation_of_classes'></a>

Classes are created as instances of `type` class, where **class_name**, **class_bases** and **class_dict** are needed:
* class_name: name of class
* class_base: class bases that the class inherits from
* class_dict: namespace containing all the objects inside the class

In [6]:
import math

In [7]:
class_name = 'Circle'
class_body = """
def __init__(self, x, y, r):
    self.x = x
    self.y = y
    self.r = r
    
def area(self):
    return math.pi * self.r**2
"""
class_bases = () # Circle class does not inherit from any class other than `object`
class_dict = {}

In [8]:
exec(class_body, globals(), class_dict)
# Create a class
Circle = type(class_name, class_bases, class_dict)

---

### Metaclasses <a name='metaclasses'></a>

`type` is by default used as metaclass when creating classes, by overriding `type`, one can use customized metaclass to control creating classes.

* Default scenario:

In [9]:
class Circle(metaclass=type):
    def __init__(self, x, y, r):
        self.x = x
        self.y = y 
        self.r = r
        
    def area(self):
        return math.pi * self.r**2

* Customized metaclass scenario:

In [10]:
class CustomType(type):
    def __new__(mcls, class_name, class_bases, class_dict):
        print(f'Using custom metaclass {mcls} to create class {class_name}.')
        cls_obj = super().__new__(mcls, class_name, class_bases, class_dict)
        cls_obj.circumference = lambda self: 2 * math.pi * self.r
        return cls_obj

In [11]:
class CircleWithCustomType(metaclass=CustomType):
    def __init__(self, x, y, r):
        self.x = x
        self.y = y 
        self.r = r
        
    def area(self):
        return math.pi * self.r**2

Using custom metaclass <class '__main__.CustomType'> to create class CircleWithCustomType.


In [12]:
c = CircleWithCustomType(0, 0, 1)
print(f'Area: {c.area()}')
print(f'Circumference: {c.circumference()}')

Area: 3.141592653589793
Circumference: 6.283185307179586


---

### Metaclass Parameters <a name='metaclass_parameters'></a>

Metaclass can be parameterized, one just needs to note that the arguments being passed in should be named arguments.

In [13]:
class MetaClass(type):
    def __new__(mcls, name, bases, cls_dict, arg1, arg2, arg3=None):
        print(arg1, arg2, arg3)
        return super().__new__(mcls, name, bases, cls_dict)

In [14]:
class MyClass(metaclass=MetaClass, arg1=1, arg2=2):
    pass

1 2 None


---

### Class Decorators <a name='class_decorators'></a>

Controlling class with metaclass is sometimes an overkill, and general functionalities can also be realized using class decorators and the implementation is very similar to decorating functions.

In [15]:
def class_logger(cls):
    print(f'Log: {cls.__name__} called.')

In [16]:
@class_logger
class Person:
    def __init__(self, name):
        self.name = name

Log: Person called.


Class decorators can also be parameterized.

In [17]:
def wrapper(name):
    def class_logger(cls):
        cls.name = name
        print(f'{cls.__name__} name: {cls.name}')
        return cls
    return class_logger

In [18]:
@wrapper("Taylor")
class Person:
    def __init__(self, name):
        self.name = name

Person name: Taylor


---

### Metaclasses vs. Class Decorators <a name='metaclasses_class_decorators'></a>

> Metaclasses:
> 1. Pros:
>   * Subclasses can inherit parent metaclass
> 2. Cons:
>   * Harder to understand
>   * Can only specify a single metaclass

> Class decorators:
> 1. Pros:
>   * More intuitive
>   * Decorators can be stacked
> 2. Cons:
>   * Subclasses cannot inherit decorator

---

### Decorator Classes <a name='decorator_classes'></a>

Instead of using a function to decorate, one can also use class for decoration by making the class callable via `__call__` method. The difference is when using function to decorate, the decorated object will also be a function; however using decorator class to decorate will give back an instance of the class.

In [19]:
from functools import wraps

def logger(fn):
    @wraps(fn)
    def wrapper(*args, **kwargs):
        print(f'Log: {fn.__name__} called.')
        return fn(*args, **kwargs)
    return wrapper

In [20]:
class Logger:
    def __init__(self, fn):
        self.fn = fn
    def __call__(self, *args, **kwargs):
        print(f'Log: {self.fn.__name__} called.')
        return self.fn(*args, **kwargs)

In [21]:
@logger
def say_hello():
    pass
say_hello()

Log: say_hello called.


In [22]:
@Logger
def say_hi():
    pass
say_hi()

Log: say_hi called.


---

### The \_\_prepare\_\_ Method <a name='the_prepare_method'></a>

`__prepare__` method is used to prepare class namespace dictionary which is then passed to `__new__` method.

In [23]:
class MyMeta(type):
    @staticmethod
    def __prepare__(name, bases, **kwargs):
        print('Calling MyMeta.__prepare__...')
        print('\tname:', name)
        print('\tbases:', bases)
        print('\tkwargs:', kwargs)
        return {'a': 100, 'b': 200}
    
    @staticmethod
    def __new__(mcls, name, bases, cls_dict, **kwargs):
        print('Calling MyMeta.__new__')
        print('\tmcls:', mcls, type(mcls))
        print('\tname:', name, type(name))
        print('\tbases:', bases, type(bases))
        print('\tcls_dict:', cls_dict, type(cls_dict))
        print('\tkwargs:', kwargs)
        return super().__new__(mcls, name, bases, cls_dict)

In [24]:
class MyClass(metaclass=MyMeta, kw1=1, kw2=2):
    pass

Calling MyMeta.__prepare__...
	name: MyClass
	bases: ()
	kwargs: {'kw1': 1, 'kw2': 2}
Calling MyMeta.__new__
	mcls: <class '__main__.MyMeta'> <class 'type'>
	name: MyClass <class 'str'>
	bases: () <class 'tuple'>
	cls_dict: {'a': 100, 'b': 200, '__module__': '__main__', '__qualname__': 'MyClass'} <class 'dict'>
	kwargs: {'kw1': 1, 'kw2': 2}
